# 引言

装饰器就是对函数的『包装』，即如果在定义函数的时候指定了使用某个装饰器对其进行『包装』，则在运行该函数时，还会同时执行装饰器中的命令或完成装饰器中的任务。

可以通过高阶函数来实现，也可以通过类来实现。通过类来实现装饰器看起来比通过高级函数来实现装饰器更加清晰。

# 通过高级函数实现装饰器

In [7]:
# 一个简单的例子

def hello(fn):
    def wrapper():
        print("Hello, %s." % fn.__name__)
        fn()
        print("Goodbye, %s." % fn.__name__)
    return wrapper

@hello
def foo():
    print('I am foo.')
    
foo()

Hello, foo.
I am foo.
Goodbye, foo.


In [8]:
# 一个网页编程的例子

def makeHtmlTag(tag, *args, **kwds):
    def real_decorator(fn):
        css_class = " class='{0}'".format(kwds["css_class"]) if "css_class" in kwds else ""
        def wrapped(*args, **kwds):
            return "<" + tag + css_class + ">" + fn(*args, **kwds) + "</" + tag + ">"
        return wrapped
    return real_decorator

@makeHtmlTag(tag="b", css_class="hold_css")
@makeHtmlTag(tag="i", css_class='italic_css')
def hello():
    return "Hello World!"

print(hello())

<b class='hold_css'><i class='italic_css'>Hello World!</i></b>


# 通过类实现装饰器

In [11]:
# 一个简单的例子

class MyDecorator(object):
    def __init__(self, fn):
        print('Inside MyDecorator.__init__()')
        self.fn = fn
        
    def __call__(self):
        self.fn()
        print('Inside MyDecorator.__call__()')
        
        
@MyDecorator
def my_func():
    print('Inside my_func()')
    
print('Finished my_func()')

my_func()

Inside MyDecorator.__init__()
Finished my_func()
Inside my_func()
Inside MyDecorator.__call__()


In [20]:
# 一个网页编程的例子

class Make_HTML_Tag(object):
    def __init__(self, tag, css_class=''):
        self._tag = tag
        self._css_class = " class='{0}'".format(css_class) if css_class != "" else ""
        
    def __call__(self, fn):
        def wrapped(*args, **kwds):
            return "<" + self._tag + self._css_class + ">" + fn(*args, **kwds) + "</" + self._tag + ">"
        return wrapped
    
@Make_HTML_Tag(tag="b", css_class="hold_css")
@Make_HTML_Tag(tag="i", css_class='italic_css')
def hello(name):
    return "Hello %s!" % name

print(hello('Sean'))

<b class='hold_css'><i class='italic_css'>Hello Sean!</i></b>
wrapped


# 装饰器的副作用

由于decorator的因素，实际被调用的函数已经变成了一个叫做`wrapped()`的函数。比如当我们调用`print(hello.__name__)`查看`hello()`函数名字的时候，将会返回`wrapped`，而不是`hello`。

Python的`functools`包中提供了一个名为`wraps`的装饰器来消除这种附着用。

In [21]:
print(hello.__name__)

wrapped


In [23]:
# 一个简单的例子

from functools import wraps

def hello(fn):
    @wraps(fn)
    def wrapped():
        print('Hello, %s!' % fn.__name__)
        fn()
        print('Hello, %s!' % fn.__name__)
    return wrapped

@hello
def foo():
    '''foo help documentation'''
    print('This is foo().')
    
foo()
print(foo.__name__)
print(foo.__doc__)

Hello, foo!
This is foo().
Hello, foo!
foo
foo help documentation


# 利用装饰器实现斐波那契数列

In [26]:
from functools import wraps

def memo(fn):
    cache = {}
    miss = object()
    
    @wraps
    def wrapped(*args):
        result = cache.get(args, miss)
        if result is miss:
            result = fn(*args)
            cache[args] = result
        return result
    return wrapped

@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

fib(5)

AttributeError: 'int' object has no attribute '__module__'

# Property属性

可以利用Property属性实现对变量读写的检查。

In [32]:
import traceback

class Student(object):
    @property
    def score(self):
        # 实现score的读取功能 (getter方法)
        return self._score
    
    @score.setter
    def score(self, value):
        # 实现score的写入功能 (setter方法)
        if not isinstance(value, int):
            raise ValueError('not int')
        elif (value<0) or (value>100):
            raise ValueError('not between 0~100')
        self._score = value
        
    @property
    def double_score(self):
        # 这是一个只读属性，因为没有定义对应的setter方法
        # 相反，score是一个读写属性
        return self._score * 2
        
s = Student()
s.score = 75
print(s.score)

75


In [30]:
try:
    s.score = 'abc'
except ValueError:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-30-628b0c99a40c>", line 2, in <module>
    s.score = 'abc'
  File "<ipython-input-27-39a7d47ba68e>", line 11, in score
    raise ValueError('not int')
ValueError: not int


In [31]:
try:
    s.score = 108
except ValueError:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-31-78e9c06cbd36>", line 2, in <module>
    s.score = 108
  File "<ipython-input-27-39a7d47ba68e>", line 13, in score
    raise ValueError('not between 0~100')
ValueError: not between 0~100


In [35]:
# double_score是一个只读属性
print(s.double_score)

try:
    s.double_score = 160
except AttributeError:
    traceback.print_exc()

150


Traceback (most recent call last):
  File "<ipython-input-35-64c039902399>", line 5, in <module>
    s.double_score = 160
AttributeError: can't set attribute


## Property属性的实现

以下代码利用描述器近似实现Property属性的基本功能。简单而言，如果一个类中实现了`__set__()`、`__get__()`、`__del__()`方法，则称这个类为一个**描述器**。

In [48]:
# 实现MyProperty
class MyProperty(object):
    def __init__(self, fget=None, fset=None, fdel = None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        
    def __get__(self, instance, cls):
        if self.fget:
            print('__get__')
            return self.fget(instance)
            
    def __set__(self, instance, value):
        if self.fset:
            print('__set__')
            return self.fset(instance, value)
            
    def __del__(self, instance):
        if self.fdel:
            return self.fdel(instance)
            
    def getter(self, fn):
        self.fget = fn
        
    def setter(self, fn):
        self.fset = fn
        
    def deler(self, fn):
        self.fdel = fn

class Student(object):
    @MyProperty
    def score(self):
        return self._score
    
    @score.setter
    def set_score(self, value):
        self._score = value
        
    
s = Student()
s.score = 95
print(s.score)

__set__
__get__
95
